*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Text Classification of MultiNLI Sentences using BERT

# Before You Start

> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

The table below provides some reference running time on different machine configurations.  

|QUICK_RUN|Machine Configurations|Running time|
|:---------|:----------------------|:------------|
|True|4 **CPU**s, 14GB memory| ~ 15 minutes|
|False|4 **CPU**s, 14GB memory| ~19.5 hours|
|True|1 NVIDIA Tesla K80 GPUs, 12GB GPU memory| ~ 3 minutes |
|False|1 NVIDIA Tesla K80 GPUs, 12GB GPU memory| ~ 1.5 hours|

If you run into CUDA out-of-memory error or the jupyter kernel dies constantly, try reducing the `BATCH_SIZE` and `MAX_LEN`, but note that model performance will be compromised. 

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = False

In [3]:
import sys
sys.path.insert(0, "../../")
import os
import json
import pandas as pd
import numpy as np
import scrapbook as sb
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

from utils_nlp.dataset.multinli import load_pandas_df
from utils_nlp.eval.classification import eval_classification
from utils_nlp.models.bert.sequence_classification import BERTSequenceClassifier
from utils_nlp.models.bert.common import Language, Tokenizer
from utils_nlp.common.timer import Timer
from pytorch_transformers import *

## Introduction
In this notebook, we fine-tune and evaluate a pretrained [BERT](https://arxiv.org/abs/1810.04805) model on a subset of the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset.

We use a [sequence classifier](../../utils_nlp/models/bert/sequence_classification.py) that wraps [Hugging Face's PyTorch implementation](https://github.com/huggingface/pytorch-pretrained-BERT) of Google's [BERT](https://github.com/google-research/bert).

In [4]:
TRAIN_DATA_USED_PERCENT = 1
TEST_DATA_USED_PERCENT = 1
NUM_EPOCHS = 1

if QUICK_RUN:
    TRAIN_DATA_USED_PERCENT = 0.01
    TEST_DATA_USED_PERCENT = 0.01
    NUM_EPOCHS = 1

if torch.cuda.is_available():
    BATCH_SIZE = 32
else:
    BATCH_SIZE = 8

DATA_FOLDER = "./temp"
BERT_CACHE_DIR = "./temp"
LANGUAGE = Language.ROBERTA
TO_LOWER = True
MAX_LEN = 150
BATCH_SIZE_PRED = 512
TRAIN_SIZE = 0.6
LABEL_COL = "genre"
TEXT_COL = "sentence1"

## Read Dataset
We start by loading a subset of the data. The following function also downloads and extracts the files, if they don't exist in the data folder.

The MultiNLI dataset is mainly used for natural language inference (NLI) tasks, where the inputs are sentence pairs and the labels are entailment indicators. The sentence pairs are also classified into *genres* that allow for more coverage and better evaluation of NLI models.

For our classification task, we use the first sentence only as the text input, and the corresponding genre as the label. We select the examples corresponding to one of the entailment labels (*neutral* in this case) to avoid duplicate rows, as the sentences are not unique, whereas the sentence pairs are.

In [5]:
df = load_pandas_df(DATA_FOLDER, "train")
df = df[df["gold_label"]=="neutral"]  # get unique sentences

In [6]:
df[[LABEL_COL, TEXT_COL]].head()

,genre,sentence1
0,government,Conceptually cream skimming has two basic dime...
4,telephone,yeah i tell you what though if you go price so...
6,travel,But a few Christian mosaics survive above the ...
12,slate,It's not that the questions they asked weren't...
13,travel,"Thebes held onto power until the 12th Dynasty,..."


The examples in the dataset are grouped into 5 genres:

In [7]:
df[LABEL_COL].value_counts()

telephone     27783
government    25784
travel        25783
fiction       25782
slate         25768
Name: genre, dtype: int64

We split the data for training and testing, and encode the class labels:

In [8]:
# split
df_train, df_test = train_test_split(df, train_size = TRAIN_SIZE, random_state=0)

/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
df_train = df_train.sample(frac=TRAIN_DATA_USED_PERCENT).reset_index(drop=True)
df_test = df_test.sample(frac=TEST_DATA_USED_PERCENT).reset_index(drop=True)

In [10]:
# encode labels
label_encoder = LabelEncoder()
labels_train = label_encoder.fit_transform(df_train[LABEL_COL])
labels_test = label_encoder.transform(df_test[LABEL_COL])

num_labels = len(np.unique(labels_train))

In [11]:
print("Number of unique labels: {}".format(num_labels))
print("Number of training examples: {}".format(df_train.shape[0]))
print("Number of testing examples: {}".format(df_test.shape[0]))

Number of unique labels: 5
Number of training examples: 78540
Number of testing examples: 52360


## Tokenize and Preprocess

Before training, we tokenize the text documents and convert them to lists of tokens. The following steps instantiate a BERT tokenizer given the language, and tokenize the text of the training and testing sets.

In [12]:
from pytorch_transformers import RobertaTokenizer

In [13]:
special_tokens = ['_start_', '_delimiter_', '_classify_']

roberta_tokenizer = RobertaTokenizer.from_pretrained(Language.ROBERTA, special_tokens=special_tokens)


In [14]:
special_tokens_ids = list(roberta_tokenizer.convert_tokens_to_ids(token) for token in special_tokens)

In [15]:
special_tokens_ids

[3, 3, 3]

In [16]:
#tokenizer = Tokenizer(roberta_tokenizer,LANGUAGE)
#tokenizer = Tokenizer(LANGUAGE, to_lower=TO_LOWER, cache_dir=BERT_CACHE_DIR)

tokens_train = [ roberta_tokenizer.encode(i) for i in list(df_train[TEXT_COL])]
tokens_test = [roberta_tokenizer.encode(i) for i in list(df_test[TEXT_COL])]

In [17]:
def preprocess(token_ids, max_len=MAX_LEN):
    temp = roberta_tokenizer.add_special_tokens_single_sentence(token_ids)
    tokens = temp[0:max_len] if len(temp)>max_len else temp + [0] * (max_len - len(temp))
    input_mask = [min(1, x) for x in tokens]
    return tokens, input_mask
    

In [18]:
tokens_train_classification = [preprocess(token_ids, MAX_LEN) for token_ids in tokens_train] 
tokens_test_classification = [preprocess(token_ids,MAX_LEN) for token_ids in tokens_test] 

In [19]:
config =  RobertaConfig.from_pretrained('roberta-base',  num_labels=num_labels)
#finetuning_task=,

In [20]:
config

{
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 5,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

In addition, we perform the following preprocessing steps in the cell below:
- Convert the tokens into token indices corresponding to the BERT tokenizer's vocabulary
- Add the special tokens [CLS] and [SEP] to mark the beginning and end of a sentence
- Pad or truncate the token lists to the specified max length
- Return mask lists that indicate paddings' positions
- Return token type id lists that indicate which sentence the tokens belong to (not needed for one-sequence classification)

*See the original [implementation](https://github.com/google-research/bert/blob/master/run_classifier.py) for more information on BERT's input format.*

## Create Model
Next, we create a sequence classifier that loads a pre-trained BERT model, given the language and number of labels.

In [21]:
from utils_nlp.models.bert.roberta_sequence_classification import RobertaSequenceClassifier
classifier = RobertaSequenceClassifier(config)

In [22]:
classifier

## Train
We train the classifier using the training examples. This involves fine-tuning the BERT Transformer and learning a linear classification layer on top of that:

In [23]:
with Timer() as t:
    classifier.fit(
        token_ids=[i[0] for i in tokens_train_classification],
        input_mask=[i[1] for i in tokens_train_classification],
        labels=labels_train,    
        num_epochs=2*NUM_EPOCHS,
        batch_size=BATCH_SIZE, 
        warmup_proportion=0.3,
        lr=2e-5,
        verbose=True,
    )    
print("[Training time: {:.3f} hrs]".format(t.interval / 3600))

Iteration:   0%|          | 1/2455 [00:04<3:04:04,  4.50s/it]

epoch:1/2; batch:1->246/2455; average training loss:1.647815


Iteration:  10%|█         | 247/2455 [05:47<52:21,  1.42s/it]

epoch:1/2; batch:247->492/2455; average training loss:1.380250


Iteration:  20%|██        | 493/2455 [11:35<45:18,  1.39s/it]

...

epoch:2/2; batch:1969->2214/2455; average training loss:0.298678


Iteration:  90%|█████████ | 2215/2455 [52:11<05:32,  1.38s/it]

epoch:2/2; batch:2215->2455/2455; average training loss:0.295982


Iteration: 100%|██████████| 2455/2455 [57:46<00:00,  1.20s/it]

[Training time: 1.931 hrs]


## Score
We score the test set using the trained classifier:

In [24]:
preds = classifier.predict(token_ids=[i[0] for i in tokens_test_classification], 
                           input_mask=[i[1] for i in tokens_test_classification], 
                           batch_size=BATCH_SIZE_PRED)

Iteration: 100%|██████████| 103/103 [09:22<00:00,  4.31s/it]


## Evaluate Results
Finally, we compute the accuracy, precision, recall, and F1 metrics of the evaluation on the test set.

In [25]:
report = classification_report(labels_test, preds, target_names=label_encoder.classes_, output_dict=True) 
accuracy = accuracy_score(labels_test, preds )
print("accuracy: {}".format(accuracy))
print(json.dumps(report, indent=4, sort_keys=True))

accuracy: 0.89207410236822
{
    "fiction": {
        "f1-score": 0.8704643370033509,
        "precision": 0.8565237870937353,
        "recall": 0.8848661800486618,
        "support": 10275
    },
    "government": {
        "f1-score": 0.9056032799687622,
        "precision": 0.909866614358572,
        "recall": 0.901379712397979,
        "support": 10292
    },
    "macro avg": {
        "f1-score": 0.8895427934405422,
        "precision": 0.8894051315286653,
        "recall": 0.8902124789219684,
        "support": 52360
    },
    "micro avg": {
        "f1-score": 0.89207410236822,
        "precision": 0.89207410236822,
        "recall": 0.89207410236822,
        "support": 52360
    },
    "slate": {
        "f1-score": 0.7734457323498419,
        "precision": 0.798487360132615,
        "recall": 0.7499270215043301,
        "support": 10277
    },
    "telephone": {
        "f1-score": 0.9945081930615707,
        "precision": 0.9950857755539672,
        "recall": 0.993931280678268

In [26]:
# for testing
sb.glue("accuracy", accuracy)
sb.glue("precision", report["macro avg"]["precision"])
sb.glue("recall", report["macro avg"]["recall"])
sb.glue("f1", report["macro avg"]["f1-score"])
